In [ ]:
import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

!pip install git+https://github.com/google/qkeras

# Tensorflow Colab file with some modifications

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [ ]:
intBits = 1
precision = 17

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid', input_shape=(32, 32, 3)))
#model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c1'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c2'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c3'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid'))
#model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        name='d1'))
#model.add(QActivation('quantized_relu({}, {})'.format(precision, intBits)))
model.add(layers.Dense(10))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images[:5000], test_labels[:5000]))

In [ ]:
import qkeras
from qkeras import *
from qkeras.utils import *

In [ ]:
model_save_quantized_weights(model)
print ("Done")

In [ ]:
model.evaluate(test_images[5000:], test_labels[5000:])

In [ ]:
### Evaluation in a loop
intBits = 1
histories = []
mymodels = []
for precision in [8, 12, 16, 24, 32]:
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c1'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c2'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c3'))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(Activation('sigmoid'))
  model.add(layers.Dense(10))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  history = model.fit(train_images, train_labels, epochs= 5 + int(3*((precision/4) - 2)), 
                    validation_data=(test_images[:5000], test_labels[:5000]), verbose=False)
  
  model_save_quantized_weights(model)
  print ("Done")

  model.evaluate(test_images[5000:], test_labels[5000:])

  mymodels.append(model)
  histories.append(history)

# Now with CORDIC

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(arr, precision, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  xarr = 1.2075*np.ones(shape=(len(arr), len(arr[0])))
  yarr = np.zeros(shape = (len(arr), len(arr[0])))

  global  lookup_arctanh

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(arr)
    
    xchange = m*sigma*2**(-i)*yarr
    ychange = sigma*2**(-i)*xarr
    arrchange = sigma*lookup_arctanh[i-1]

    xarr -= xchange
    yarr += ychange
    arr -= arrchange
    
  ex = xarr + yarr

  #temp = quantized_bits(precision, 1, alpha=1)(1 + ex)
  return quantized_bits(precision,1, alpha=1)(ex/(1+ex))

In [ ]:
qmodel = 0 

def epicGeneratePredictions(indices, precision, iterations):
  global qmodel
  x_test = test_images
  from keras import backend as K
  get_sixth_layer_output = K.function([qmodel.layers[0].input],
                                    [qmodel.layers[6].output])
 
  layer6_output = get_sixth_layer_output([x_test[indices[0]: indices[1]]])[0]

  #layer6_output = layer6_output[indices[0]: indices[1]]

  layer7_output = modifiedCordicTanh(layer6_output, precision, iterations)

  input_shape = qmodel.layers[8].get_input_shape_at(0)
  layer_input = Input(shape=(64))
  x = layer_input
  x = qmodel.layers[8](x)
  qm4 = Model(layer_input, x)

  predictions = np.array(qm4.predict(layer7_output))

  #predictions = modifiedCordicTanh(predictions, precision, iterations)

  a = predictions
  return (a == a.max(axis=1)[:,None]).astype(int)

In [ ]:
def modelMaker(precision, intBits, epoch=10):
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c1'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c2'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c3'))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(Activation('sigmoid'))
  model.add(layers.Dense(10))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  history = model.fit(train_images, train_labels, epochs= epoch, 
                    validation_data=(test_images[:5000], test_labels[:5000]), verbose=False)
  
  model_save_quantized_weights(model)
  return model

## 8bit

In [ ]:
qmodel = modelMaker(9, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([5000, 10000], 9, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[5000:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(5000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

## 12 bit

In [ ]:
qmodel = modelMaker(13, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([5000, 10000], 13, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[5000:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(5000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

## 16 bit

In [ ]:
qmodel = modelMaker(17, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([5000, 10000], 17, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[5000:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(5000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

## 24 bit

In [ ]:

for epochs in range(8, 15):
  qmodel = modelMaker(33, 1, epochs)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([5000, 10000], 33, iter))
    print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[5000:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(5000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

## 32 Bit

In [ ]:
qmodel = modelMaker(33, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([5000, 10000], 33, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[5000:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(5000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy